In [1]:
import numba
import time

import pandas as pd
import numpy as np



In [58]:
# data reference https://github.com/zygmuntz/goodbooks-10k/blob/master/tags.csv
ratings = pd.read_csv("./data/ratings.csv")


In [3]:
ratings[:2]

,user_id,book_id,rating
0,1,258,5
1,2,4081,4


In [59]:
# create user-item matrix as numpy array
n_users = user_book_matrix.shape[0]
n_items = user_book_matrix.shape[1]
ratings_np = np.zeros((n_users, n_items))
for row in ratings.itertuples():
    ratings_np[row[1]-1, row[2]-1] = row[3]
ratings_np

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [4., 5., 0., ..., 0., 0., 0.],
       [4., 5., 0., ..., 0., 0., 0.],
       [4., 5., 4., ..., 0., 0., 0.]])

In [60]:
# calculate the fraction of user-item values filled
sparsity = float(len(ratings_np.nonzero()[0]))
sparsity /= (ratings_np.shape[0] * ratings_np.shape[1])
sparsity *= 100
print("Sparsity: {:4.2f}%".format(sparsity))

Sparsity: 1.12%


In [61]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    """TODO"""
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [ ]:
# calculate the similarity matrix
similarity = fast_similarity(ratings_np, "item")

In [84]:
def find_k_most_similar(similarity, book_id, k=5):
    """TODO"""
    all_similarities = similarity[:, book_id-1]
    # prevent that highest similarity is with itself
    all_similarities[book_id - 1] = 0
    # k indexes with highes similarity to book_id
    most_similar = np.flip(np.argpartition(all_similarities, -k)[-k:]) + 1
    return most_similar


In [81]:
find_k_most_similar(similarity, 1, 5)

array([17, 20,  2,  3, 12])

In [91]:
def make_item_based_recommendation(books_liked, n_rec, similarity):
    """TODO"""
    # for each book find most similar ones
    book_similarity = {}
    for book_id in books_liked:
        most_similar = find_k_most_similar(similarity, book_id)
        # add similar book ids and similarities to dict
        for similar_book_id in most_similar:
            book_similarity[similar_book_id] = similarity[book_id-1][similar_book_id-1]

    # sort the dict by the similarity
    sorted_book_similarity = dict(sorted(book_similarity.items(), key=lambda item: item[1], reverse=True))
    print(sorted_book_similarity)
    recommendations = list(sorted_book_similarity.keys())[:n_rec]
    return recommendations

In [93]:
# obtain 10 recommendations based on movies one and two
make_item_based_recommendation([1, 2], 10, similarity)

{18: 0.6803923969196116, 23: 0.6728205638365854, 24: 0.6692005033391328, 21: 0.6574468707948864, 27: 0.6520871016336011}


[18, 23, 24, 21, 27]